# Calling Azure OpenAI with your customized Data 

The following notebook will show you an example of calling an Azure OpenAI model that uses your custom data (previously done following these instructions https://microsoftlearning.github.io/mslearn-openai/Instructions/Labs/06-use-own-data.html )

In [12]:
import openai
import os
import requests

# Azure OpenAI on your own data is only supported by the 2023-08-01-preview API version
openai.api_version = "2023-08-01-preview"



Load Azure OpenAI settings from the GitHub Codespace secrets and **replace by your own model deployment name**

In [13]:
openai.api_type: str = "azure"
# openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
# openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")


# Add your deployment ID here
deployment_id = "gpt-35-turbo-unai" 

search_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")

Add the name of the index created in Azure Search for the custom data (it keeps the vector embeddings of th data). The Azure OpenAI playground shows you the name of the index

![Index name](custom-aoai-reply.png)

In [3]:
search_index_name = "custom-data-margiestravel"; # Add your Azure Cognitive Search index name here

Setup OpenAI SDK to use your own data

In [5]:

def setup_byod(deployment_id: str) -> None:
    """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.

    :param deployment_id: The deployment ID for the model to use with your own data.

    To remove this configuration, simply set openai.requestssession to None.
    """

    class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):

        def send(self, request, **kwargs):
            request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
            return super().send(request, **kwargs)

    session = requests.Session()

    # Mount a custom adapter which will use the extensions endpoint for any call using the given `deployment_id`
    session.mount(
        prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
        adapter=BringYourOwnDataAdapter()
    )

    openai.requestssession = session

setup_byod(deployment_id)




Test your custom completions!

In [14]:
message = "What are some facts about New York?"  

completion = openai.ChatCompletion.create(
    messages=[{"role": "user", "content": message}],
    deployment_id=deployment_id,
    dataSources=[  # camelCase is intentional, as this is the format the API expects
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": search_endpoint,
                "key": search_key,
                "indexName": search_index_name,
            }
        }
    ]
)
print(completion)


{
  "id": "b21d70ab-5b60-4030-9e10-0ad06fcdcf7a",
  "model": "gpt-35-turbo",
  "created": 1695735807,
  "object": "extensions.chat.completion",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "New York City is the most populous city in the United States with an estimated 2015 population of 8,550,405 distributed over a land area of about 302.6 square miles [doc2]. It is also the most densely populated major city in the United States [doc2]. New York City is the center of the New York metropolitan area, one of the most populous urban agglomerations in the world [doc2]. It is a global power city, exerting a significant impact upon commerce, finance, media, art, fashion, research, technology, education, and entertainment [doc2]. It is home to the headquarters of the United Nations and has been described as the cultural and financial capital of the world [doc2].",
        "end_turn": true,
        "cont